In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as datetime
import pickle

from loadandconvert import loadCSIfluxperiod, deltatogas,f_gg_water, f_gg_ice,alpha18O_Maj,alphaD_MN,nearest,exportdf

#for plotting
import mpl_axes_aligner.align as align
from loadandconvert import fontsize,plot1,plot2
fontsize()
import matplotlib.dates as mdates



In [62]:


importnew=0
if importnew:
    #%% import DC samples of experiment 3


    # I think the samples were taken consecutively. I.e. 
    # UL=0.5cm
    # 1=05to1cm
    # 2=1to2cm
    # 4=2to4cm

    Dc3=pd.read_csv('/Users/swa048/Documents/OneDrive/Dokumente/EGRIP/EGRIP2018/EGRIP_Data/EGRIP2018_DC_samples.csv',usecols=(0,1,2,3,4,5), na_values=['NaN'],dtype={'DC_EGRIP_2018_newsampleww': object})
    Dc3.rename(inplace=True, columns={
        'DC_EGRIP_2018_newsampleww': 'samplename',
        'DC_EGRIP_2018_d18O_meanww': 'd18O',
        'DC_EGRIP_2018_d18O_SDww': 'd18O_SD',
        'DC_EGRIP_2018_ddeut_meanww': 'dD',
        'DC_EGRIP_2018_ddeut_SDww': 'dD_SD',
        'DC_EGRIP_2018_dexcessww': 'dex',
    })

    Dc3_Ident=Dc3.samplename.str.split('_', expand=True)
    Dc3_Ident.columns=['Exp','date','time','box','Slevel']

    Dc3=pd.concat([Dc3,Dc3_Ident],axis=1)
    Dc3['timestamp'] = pd.to_datetime(Dc3['date'] + " " + Dc3['time'])
    #Dc3.delete(106)
    Dc3.index=Dc3.timestamp


    # the individual boxes with each level
    dfBox1=Dc3[Dc3.box=='1']
    dfBox2=Dc3[Dc3.box=='2']
    dfBox3=Dc3[Dc3.box=='3']
    dfVal=Dc3[Dc3.box=='VAL']
    dfVal.index = Val.timestamp
    dfVal=dfVal.rename(columns={'Slevel': 'DOY','d18O':'d18OVal','dD':'dDVal','dex':'dexVal'})
    dfVal.loc[:,'DOY'] = dfVal.index.dayofyear

    # the levels individually grouped
    #05cm
    dfUL=Dc3[Dc3.Slevel=='UL']
    dfULmean=dfUL.loc[:,['d18O','dD','dex']].groupby(dfUL.index).agg([np.mean,np.std])
    #testcol = dfULmean.columns.map('_'.join)+'_UL'#.str.strip('|')
    dfULmean.columns = dfULmean.columns.map(''.join)+'_05cm'#.str.strip('|')

    #1cm
    df1cm=Dc3[Dc3.Slevel=='1']
    df1cmmean=df1cm.loc[:,['d18O','dD','dex']].groupby(df1cm.index).agg([np.mean,np.std])
    df1cmmean.columns = df1cmmean.columns.map(''.join)+'_05to1cm'#.str.strip('|')


    #2cm
    df2cm=Dc3[Dc3.Slevel=='2']
    df2cmmean=df2cm.loc[:,['d18O','dD','dex']].groupby(df2cm.index).agg([np.mean,np.std])
    df2cmmean.columns = df2cmmean.columns.map(''.join)+'_1to2cm'#.str.strip('|')


    #4cm
    df4cm=Dc3[Dc3.Slevel=='4']
    df4cmmean=df4cm.loc[:,['d18O','dD','dex']].groupby(df4cm.index).agg([np.mean,np.std])
    df4cmmean.columns = df4cmmean.columns.map(''.join)+'_2to4cm'#.str.strip('|')


    # all in one dataframe with column names
    dfDC3=pd.concat([dfULmean,df1cmmean,df2cmmean,df4cmmean,dfVal.loc[:,['d18OVal','dDVal','dexVal']]],axis=1)

    with open('/Users/swa048/forServer/Snow/Isotopes/2018/DiurnalCycle/Data/dfDC3.pkl', 'wb') as f:
        pickle.dump(dfDC3, f)    
else:
    with open('/Users/swa048/forServer/Snow/Isotopes/2018/DiurnalCycle/Data/dfDC3.pkl', 'rb') as f:
        dfDC3=pickle.load(f)  